# Interact COMET:

Langchain - Comet: https://www.comet.com/docs/v2/integrations/third-party-tools/langchain/

I will use the code to chat_with_news: http://localhost:8888/notebooks/PycharmProjects/tradingGPT/scripts/chat_with_news.ipynb

First, prepare the data I used in this notebook.

In [ ]:
from financial_data import FinancialDataFinnHub
import os


# Create Finnhub object
symbol = "NVDA"
api_key = os.environ.get("FINHUB_API_KEY")
nvda = FinancialDataFinnHub(symbol, api_key)

# Use FinancialHub to load news url
news_urls = nvda.company_news['url']
urls = [news_urls.iloc[i][0] for i in range(len(news_urls))]


# langchain UnstructuredURLLoader to load news
from langchain.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

# Remove news if the content is not reacheable. 
for i, d in enumerate(data):
    p_content = d.page_content[:22]
    if p_content == 'Javascript is Disabled':
        data.pop(i)
print(f"You got {len(data)} news to read about {symbol}. ")

# Create `OpenAIEmbeddings` model
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Split text with RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

# Store vectors in Chroma 
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)

In [ ]:
from langchain.agents import initialize_agent, load_tools
from langchain.callbacks import CometCallbackHandler, StdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
from langchain.llms import OpenAI

comet_callback = CometCallbackHandler(
    project_name="comet-example-langchain",
    complexity_metrics=True,
    stream_logs=True,
    tags=["agent"],
)
manager = CallbackManager([StdOutCallbackHandler(), comet_callback])
llm = OpenAI(temperature=0.9, callback_manager=manager, verbose=True)

tools = load_tools(["serpapi", "llm-math"], llm=llm, callback_manager=manager)
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    callback_manager=manager,
    verbose=True,
)
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)
comet_callback.flush_tracker(agent, finish=True)

In [1]:
from rouge_score import rouge_scorer

from langchain.callbacks import CometCallbackHandler, StdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate


class Rouge:
    def __init__(self, reference):
        self.reference = reference
        self.scorer = rouge_scorer.RougeScorer(["rougeLsum"], use_stemmer=True)

    def compute_metric(self, generation, prompt_idx, gen_idx):
        prediction = generation.text
        results = self.scorer.score(target=self.reference, prediction=prediction)

        return {
            "rougeLsum_score": results["rougeLsum"].fmeasure,
            "reference": self.reference,
        }


reference = """
The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building.
It was the first structure to reach a height of 300 metres.

It is now taller than the Chrysler Building in New York City by 5.2 metres (17 ft)
Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France .
"""
rouge_score = Rouge(reference=reference)

template = """Given the following article, it is your job to write a summary.
Article:
{article}
Summary: This is the summary for the above article:"""
prompt_template = PromptTemplate(input_variables=["article"], template=template)

comet_callback = CometCallbackHandler(
    project_name="comet-example-langchain",
    complexity_metrics=False,
    stream_logs=True,
    tags=["custom_metrics"],
    custom_metrics=rouge_score.compute_metric,
)
manager = CallbackManager([StdOutCallbackHandler(), comet_callback])
llm = OpenAI(temperature=0.9, callback_manager=manager, verbose=True)

synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

test_prompts = [
    {
        "article": """
                 The tower is 324 metres (1,063 ft) tall, about the same height as
                 an 81-storey building, and the tallest structure in Paris. Its base is square,
                 measuring 125 metres (410 ft) on each side.
                 During its construction, the Eiffel Tower surpassed the
                 Washington Monument to become the tallest man-made structure in the world,
                 a title it held for 41 years until the Chrysler Building
                 in New York City was finished in 1930.

                 It was the first structure to reach a height of 300 metres.
                 Due to the addition of a broadcasting aerial at the top of the tower in 1957,
                 it is now taller than the Chrysler Building by 5.2 metres (17 ft).

                 Excluding transmitters, the Eiffel Tower is the second tallest
                 free-standing structure in France after the Millau Viaduct.
                 """
    }
]
print(synopsis_chain.apply(test_prompts))
comet_callback.flush_tracker(synopsis_chain, finish=True)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/masahiroichigo/comet-example-langchain/7ead0637f67044db944e05cdf297073e

COMET WARNING: The comet_ml callback is currently in beta and is subject to change based on updates to `langchain`. Please report any issues to https://github.com/comet-ml/issue_tracking/issues with the tag `langchain`.


[{'text': ' The Eiffel Tower is the tallest structure in Paris, standing at 324m (1,063ft) tall. When constructed, it was the tallest man-made structure in the world until the Chrysler Building in New York City was built in 1930. In 1957, an aerial was added to the top of the tower, making it taller than the Chrysler Building by 5.2m (17ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.'}]


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/masahiroichigo/comet-example-langchain/7ead0637f67044db944e05cdf297073e
COMET INFO:   Metrics:
COMET INFO:     openai_completion_tokens : 110
COMET INFO:     openai_prompt_tokens     : 224
COMET INFO:     openai_total_tokens      : 334
COMET INFO:     rougeLsum_score_25%      : 0.5352112676056339
COMET INFO:     rougeLsum_score_50%      : 0.5352112676056339
COMET INFO:     rougeLsum_score_75%      : 0.5352112676056339
COMET INFO:     rougeLsum_score_count    : 1.0
COMET INFO:     rougeLsum_score_max      : 0.5352112676056339
COMET INFO:     rougeLsum_score_mean     : 0.5352112676056339
COMET INFO:     rougeLsum_score_min      : 

In [ ]:
pip install rouge_score

In [ ]:

from comet_ml import Experiment
          
experiment = Experiment(
  api_key = os.environ.get("COMET_API_KEY"),
  project_name = "general",
  workspace="masahiroichigo"
)

In [ ]:
experiment.end()